# Auto Outlier Detection (UIUC)

## User Inputs

In [1]:
PrimaryTableFileLocation = 'quote_level.csv'

## Required Imports

In [2]:
import pandas as pd
import numpy as np
import math
import json
import time
import random
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.compose import ColumnTransformer

## Required Functions

In [3]:
initial = time.time()
def load_json_dict(fname):
    with open(fname) as json_file:
        returndict = json.load(json_file)
    return returndict

def cont_agg(df, continuouscols, key_cols):
    """
    Function to aggregate continuous variables
    Purposes:
        Determine min, max, mean for each continuous variable
        Determine % of total for boolean missing value indicators
    """
    # Construct return dataframe
    ReturnDF = df[key_cols].drop_duplicates()
    # Contruct pivot containing min, max, mean for every continuous variable
    curpivot = pd.pivot_table(df[list(set(key_cols)|set(continuouscols))], values=continuouscols, dropna = False, index=key_cols, aggfunc=[min, max, np.mean])
    curpivot.columns = curpivot.columns.map('_'.join)
    curpivot = curpivot.reset_index()
    ReturnDF = ReturnDF.merge(curpivot, on=key_cols, how='inner')
    del curpivot
    # Contruct pivot containing % missing for every missing value indicator
    missingvalcols = [col for col in df.columns if '_missing_ind' in col]
    missingpivot = pd.pivot_table(df[list(set(key_cols)|set(missingvalcols))], values=missingvalcols, dropna = False, index=key_cols, aggfunc=[np.mean])
    missingpivot.columns = missingpivot.columns.map('_'.join)
    missingpivot = missingpivot.reset_index()
    ReturnDF = ReturnDF.merge(missingpivot, on=key_cols, how='inner')
    del missingpivot
    return ReturnDF



def aggregate_discrete(df,key_col,encodeddict):
    #key_col refers to trans_tracking_nbr
    output_df = df.groupby(key_col).agg([np.mean])
    output_df.columns = output_df.columns.droplevel(1)
    
    aggregated_dict = {}
    for col in df.columns:
        if encodeddict[col] == 'key':
            aggregated_dict[col] = 'key'
        else:
            aggregated_dict[col] = 'numeric'
    
            
    return output_df, aggregated_dict

def get_means_by_year(df, columns, year_col):
    means_by_year = df[list(set(columns) | set([year_col]))].groupby(year_col).agg([np.mean])
    means_by_year.columns = means_by_year.columns.droplevel(1) #rename columns
    means_by_year = means_by_year.reset_index()
    means_by_year.columns = [CurCol+'_imputemean' if CurCol != year_col else CurCol for CurCol in means_by_year.columns]
    return means_by_year

def get_means_overall(df, columns):
    means_overall = df[columns].agg([np.mean])
    means_overall.columns = [CurCol+'_imputemean' for CurCol in means_overall.columns]
    return means_overall

def impute_missing_by_year(df, columns, year_col, means_by_year):
    merged_data = df.merge(means_by_year, on = [year_col], how = "inner")
    for col in columns:
        #print(col)
        merged_data[col+'_missing_ind'] = np.where(merged_data[col].isnull(),1,0)
        merged_data[col] = np.where(merged_data[col].isnull(),merged_data[col+'_imputemean'],merged_data[col])
    merged_data.drop([col for col in merged_data.columns if '_imputemean' in col], inplace = True, axis = 1)
    return merged_data

def impute_missing_overall(df, columns, means_overall):
    merged_data = df.merge(means_overall, how = "cross")
    for col in columns:
        merged_data[col] = np.where(merged_data[col].isnull(),merged_data[col+'_imputemean'],merged_data[col])
    merged_data.drop([col for col in merged_data.columns if '_imputemean' in col], inplace = True, axis = 1)
    return merged_data

def normalize_data(df, columns, year_col = None):
    if year_col is not None:
        df_grouped = df.groupby(year_col)
        df[columns] =df_grouped[columns].transform(lambda x: scale(x.astype(float)))
    else:
        df[columns]= df[columns].transform(lambda x: scale(x.astype(float)))
    return df

#df is a dataframe that contains all the keys and the columns that need to be one hot encoded.
#identifier is a dictionary that signifies if the column in the dataframe is a "key" or a "discrete"
def One_Hot_Encode(df, discretecols, keycols):
    encoded_df = pd.DataFrame() #intialize the dataframe and the dicionary to return
    encoded_dict = {}
    
    for col in df.columns: #iterate through each column in the input dataframe
        #print(col)
        if col in keycols: #copy over column if its a key and store it in the dictionary as a key
            temp_df = df[[col]]
            encoded_df[temp_df.columns] = temp_df
            encoded_dict[col] = "key" 
        elif col in discretecols: #one hot encode column if its discrete and store each one hot encoded column in the dictionary as a numeric 
            df[col] = df[col].str.replace(" ","").str.lower().replace('nan', np.nan)
            temp_df = pd.get_dummies(df[[col]], dummy_na = True)
            #print(temp_df.columns)
            encoded_df[temp_df.columns] = temp_df
            for c in temp_df.columns:
                encoded_dict[c] = "numeric"
    #DEBUGGING
    #print(encoded_df)
    #print(encoded_dict)
    
    return encoded_df, encoded_dict

# removes special characters from a column
# def remove_special(df, column):
#     df[column] = df[column].str.lower().replace('\s+', '', regex=True).str.strip().replace('[#,@,&,-,/,*,--]', '')
def remove_special(x):
    return str(x).lower().replace('\s+', '').strip().replace('[#,@,&,-,/,*,--]', '')
    
def count_distinct(df, key_cols, returncolname):
    #select cols
    df_unique = df[key_cols].copy()
    df_unique[returncolname] = 1
    df_unique = df_unique.groupby(by = key_cols).count()
#     df_unique_group = df_unique.groupby(by = key_cols).count()
#     df_unique_group.reset_index(inplace = True)
#     df_unique_group[returncolname] = df_unique_group[distinct_col_list[0]]
#     return df_unique_group[list(set(key_cols)|set([returncolname]))]
    return df_unique.reset_index()

## Fully Pre-Process Quote Dataset (Primary File)

In [4]:
# Load json associated with primary file (ie. primarydict = load_json_dict('primary_dict.json'))
quote_dict = load_json_dict('col_types/quote_level_dict.json')
display(quote_dict)

{'qtever': 'continuous',
 'premium': 'continuous',
 'age_youngest_driver': 'continuous',
 'pd_limit': 'continuous',
 'newest_vehicle': 'continuous',
 'oldest_vehicle': 'continuous',
 'vehicles': 'continuous',
 'age_oldest_driver': 'continuous',
 'drivers': 'continuous',
 'chargeable13': 'continuous',
 'chargeable45': 'continuous',
 'cbrscore': 'continuous',
 'cbr': 'continuous',
 'effdate': 'other',
 'zipcode': 'other',
 'open_year_month': 'other',
 'open_day_current': 'other',
 'open_month_current': 'other',
 'open_year_current': 'other',
 'rateset': 'other',
 'duplicate_clientid?': 'other',
 'territory': 'other',
 'district': 'other',
 'policy_state': 'other',
 'office_state': 'other',
 'requote': 'discrete',
 'policy_term': 'discrete',
 'tenuretype': 'discrete',
 'tenuresub': 'discrete',
 'agent_type': 'discrete',
 'office_name': 'discrete',
 'quoteloc': 'discrete',
 'bill_mode': 'discrete',
 'company': 'discrete',
 'rateplan': 'discrete',
 'upp': 'discrete',
 'occupation_discount':

In [5]:
# Import quote data from PrimaryTableFileLocation
quote_data = pd.read_csv(PrimaryTableFileLocation)
try:
    quote_data.drop("SampleFilter", axis = 1, inplace=True)
except:
    pass
finally:
    display(quote_data)
set(quote_data.columns) - set(quote_dict)
set(quote_dict) - set(quote_data.columns)

c:\users\id75147\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (0,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,hhid,qtever,requote,trans_tracking_nbr,open_year_current,open_month_current,open_day_current,open_year_month,effdate,premium,...,prior_bi,married,male,female,mpd,farm_use,large_farm_truck,semi_farm_truck,cbrscore,mulq
0,1,1,N,806186,2014,7,3,201407,20140710,668.38,...,NaN,Y,Y,Y,AutoHomeLife,N,N,N,0,N
1,1000002,1,N,67166484,2018,6,12,201806,20180619,351.85,...,250000/500000,N,N,Y,AutoHome,N,N,N,819,N
2,1000003,1,N,71322809,2018,7,19,201806,20180702,341.45,...,100000/300000,N,Y,N,AutoHome,N,N,N,0,N
3,1000004,1,N,67166377,2018,6,12,201806,20180612,1892.81,...,50000/100000,N,N,Y,AutoTenant,N,N,N,467,N
4,1000005,1,N,67166542,2018,6,12,201806,20180714,1987.31,...,25000/50000,Y,Y,Y,NaN,N,N,N,626,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717455,Q978792001,1,N,47433200,2017,10,7,201710,20171007,908.70,...,100000/300000,N,N,Y,AutoTenant,N,N,N,522,N
717456,Q98232001,1,N,38302236,2017,5,8,201705,20170526,957.70,...,50000/100000,Y,Y,Y,AutoHome,N,N,N,0,N
717457,Q987232001,1,N,38287210,2017,5,8,201705,20170515,845.85,...,25000/50000,N,Y,N,NaN,N,N,N,0,N
717458,Q988232001,1,N,38302245,2017,5,8,201705,20170515,1195.91,...,100000/300000,N,N,Y,NaN,N,N,N,0,N


{'Year', 'duplicate_clientid?'}

In [6]:
# Apply ad-hoc adjustments (ie. reduce number of discrete levels)
quote_data["Year"] = quote_data["effdate"].apply(lambda X: str(X)[0:4])
# Construct credit score indicators (insufficient credit and not hit)
quote_data["cbr_insufficientcredit_ind"] = quote_data["cbr"].apply(lambda x: 1 if str(x).lower() == 'i' else 0)
quote_dict["cbr_insufficientcredit_ind"] = 'continuous'
quote_data["cbr_nohit_ind"] = quote_data["cbr"].apply(lambda x: 1 if str(x).lower() == 'n' else 0)
quote_dict["cbr_nohit_ind"] = 'continuous'
quote_data["cbr"] = quote_data["cbr"].apply(lambda x: x if str(x).lower() not in ['i','n'] else np.nan)
# Convert credit score data type to continuous (1a->1.01, 1b->1.02, ... 2->2, I->NAN, N->NAN)
quote_data["cbr"] = quote_data["cbr"].apply(lambda x: x if len(str(x)) != 2 else str(float(list(x)[0]) + 0.01*(ord(list(x)[1].lower()) - ord('a') + 1)))
quote_data["comp_ded"] = quote_data["comp_ded"].replace("None", 0)
quote_data["coll_ded"] = quote_data["coll_ded"].replace("None", 0)

In [7]:
# Cast columns to the correct type
for CurColumn in quote_dict.keys():
    if quote_dict[CurColumn] == 'discrete':
        quote_data[CurColumn] = quote_data[CurColumn].apply(str)
    elif quote_dict[CurColumn] == 'continuous':
        # replace "None" strings with np.nan value
        quote_data[CurColumn] = quote_data[CurColumn].replace("None", np.nan)
        quote_data[CurColumn] = quote_data[CurColumn].apply(float)
    elif quote_dict[CurColumn] == 'key':
        quote_data[CurColumn] = quote_data[CurColumn].apply(str)
    else:
        pass
display(quote_data)

,hhid,qtever,requote,trans_tracking_nbr,open_year_current,open_month_current,open_day_current,open_year_month,effdate,premium,...,female,mpd,farm_use,large_farm_truck,semi_farm_truck,cbrscore,mulq,Year,cbr_insufficientcredit_ind,cbr_nohit_ind
0,1,1.0,N,806186,2014,7,3,201407,20140710,668.38,...,Y,AutoHomeLife,N,N,N,0.0,N,2014,0.0,0.0
1,1000002,1.0,N,67166484,2018,6,12,201806,20180619,351.85,...,Y,AutoHome,N,N,N,819.0,N,2018,0.0,0.0
2,1000003,1.0,N,71322809,2018,7,19,201806,20180702,341.45,...,N,AutoHome,N,N,N,0.0,N,2018,0.0,0.0
3,1000004,1.0,N,67166377,2018,6,12,201806,20180612,1892.81,...,Y,AutoTenant,N,N,N,467.0,N,2018,0.0,0.0
4,1000005,1.0,N,67166542,2018,6,12,201806,20180714,1987.31,...,Y,nan,N,N,N,626.0,N,2018,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717455,Q978792001,1.0,N,47433200,2017,10,7,201710,20171007,908.70,...,Y,AutoTenant,N,N,N,522.0,N,2017,0.0,0.0
717456,Q98232001,1.0,N,38302236,2017,5,8,201705,20170526,957.70,...,Y,AutoHome,N,N,N,0.0,N,2017,0.0,0.0
717457,Q987232001,1.0,N,38287210,2017,5,8,201705,20170515,845.85,...,N,nan,N,N,N,0.0,N,2017,0.0,0.0
717458,Q988232001,1.0,N,38302245,2017,5,8,201705,20170515,1195.91,...,Y,nan,N,N,N,0.0,N,2017,0.0,0.0


In [8]:
# Continuous variables -- impute missing values
# Extract list of continuous columns from dictionary
continuouscols = [col for col in quote_dict.keys() if quote_dict[col] == 'continuous']
# Construct dataframe containing means by year (for imputation)
#display(continuouscols)
########################
# Impute means by year #
########################
means_by_year = get_means_by_year(quote_data, continuouscols, "Year")
# Impute missing continuous variables
quote_data = impute_missing_by_year(quote_data, continuouscols, "Year", means_by_year)
del means_by_year
##################################################
# Impute means (overall)                         #
# ... incase column wasn't used for a given year #
##################################################
# Construct dataframe containing overall means (for imputation)
means_overall = get_means_overall(quote_data, continuouscols)
# Impute missing continuous variables
quote_data = impute_missing_overall(quote_data, continuouscols, means_overall)
del means_overall

display(quote_data)

,hhid,qtever,requote,trans_tracking_nbr,open_year_current,open_month_current,open_day_current,open_year_month,effdate,premium,...,oldest_vehicle_missing_ind,vehicles_missing_ind,age_oldest_driver_missing_ind,drivers_missing_ind,chargeable13_missing_ind,chargeable45_missing_ind,cbrscore_missing_ind,cbr_missing_ind,cbr_insufficientcredit_ind_missing_ind,cbr_nohit_ind_missing_ind
0,1,1.0,N,806186,2014,7,3,201407,20140710,668.38,...,0,0,0,0,0,0,0,1,0,0
1,100007,1.0,N,3437185,2014,12,3,201412,20141203,474.64,...,0,0,0,0,0,0,0,1,0,0
2,100008,1.0,N,3440190,2014,12,3,201412,20141203,1219.78,...,0,0,0,0,0,0,0,1,0,0
3,100009,1.0,N,3441186,2014,12,3,201412,20141203,795.43,...,0,0,0,0,0,0,0,1,0,0
4,100010,1.0,N,3442186,2014,12,3,201412,20141203,355.95,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717455,Q978792001,1.0,N,47433200,2017,10,7,201710,20171007,908.70,...,0,0,0,0,0,0,0,0,0,0
717456,Q98232001,1.0,N,38302236,2017,5,8,201705,20170526,957.70,...,0,0,0,0,0,0,0,1,0,0
717457,Q987232001,1.0,N,38287210,2017,5,8,201705,20170515,845.85,...,0,0,0,0,0,0,0,1,0,0
717458,Q988232001,1.0,N,38302245,2017,5,8,201705,20170515,1195.91,...,0,0,0,0,0,0,0,1,0,0


In [9]:
# Construct curated dataset
keepkeylist = list(set([col for col in quote_dict.keys() if quote_dict[col] == 'key']) -set(['hhid']))
#keepkeylist = ["trans_tracking_nbr", "Year"]
display(keepkeylist)
Curated_Quote = quote_data[keepkeylist].drop_duplicates()
Curated_Quote

['trans_tracking_nbr', 'Year']

,trans_tracking_nbr,Year
0,806186,2014
1,3437185,2014
2,3440190,2014
3,3441186,2014
4,3442186,2014
...,...,...
717455,47433200,2017
717456,38302236,2017
717457,38287210,2017
717458,38302245,2017


In [10]:
# Process continuous variables
cont_df = cont_agg(quote_data, continuouscols, keepkeylist)
Curated_Quote = Curated_Quote.merge(cont_df, on=keepkeylist, how='inner')
del cont_df
Curated_Quote

,trans_tracking_nbr,Year,min_age_oldest_driver,min_age_youngest_driver,min_cbr,min_cbr_insufficientcredit_ind,min_cbr_nohit_ind,min_cbrscore,min_chargeable13,min_chargeable45,...,mean_cbrscore_missing_ind,mean_chargeable13_missing_ind,mean_chargeable45_missing_ind,mean_drivers_missing_ind,mean_newest_vehicle_missing_ind,mean_oldest_vehicle_missing_ind,mean_pd_limit_missing_ind,mean_premium_missing_ind,mean_qtever_missing_ind,mean_vehicles_missing_ind
0,806186,2014,29.0,28.0,3.256082,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3437185,2014,53.0,53.0,3.256082,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3440190,2014,34.0,34.0,3.256082,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3441186,2014,48.0,48.0,3.256082,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3442186,2014,54.0,54.0,3.256082,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717455,47433200,2017,67.0,67.0,9.000000,0.0,0.0,522.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
717456,38302236,2017,40.0,33.0,3.501901,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
717457,38287210,2017,26.0,26.0,3.501901,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
717458,38302245,2017,22.0,22.0,3.501901,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Process discrete variables
discretecols = [col for col in quote_dict.keys() if quote_dict[col] == 'discrete']
encoded_df, encoded_dict = One_Hot_Encode(quote_data, discretecols, keepkeylist)
encoded_df

,requote_n,requote_y,requote_nan,trans_tracking_nbr,policy_term_12,policy_term_6,policy_term_nan,tenuretype_new,tenuretype_nan,tenuresub_new,...,large_farm_truck_n,large_farm_truck_y,large_farm_truck_nan,semi_farm_truck_n,semi_farm_truck_y,semi_farm_truck_nan,mulq_n,mulq_y,mulq_nan,Year
0,1,0,0,806186,0,1,0,1,0,1,...,1,0,0,1,0,0,1,0,0,2014
1,1,0,0,3437185,0,1,0,1,0,1,...,1,0,0,1,0,0,1,0,0,2014
2,1,0,0,3440190,0,1,0,1,0,1,...,1,0,0,1,0,0,1,0,0,2014
3,1,0,0,3441186,0,1,0,1,0,1,...,1,0,0,1,0,0,1,0,0,2014
4,1,0,0,3442186,0,1,0,1,0,1,...,1,0,0,1,0,0,1,0,0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717455,1,0,0,47433200,0,1,0,1,0,1,...,1,0,0,1,0,0,1,0,0,2017
717456,1,0,0,38302236,0,1,0,1,0,1,...,1,0,0,1,0,0,1,0,0,2017
717457,1,0,0,38287210,0,1,0,1,0,1,...,1,0,0,1,0,0,1,0,0,2017
717458,1,0,0,38302245,0,1,0,1,0,1,...,1,0,0,1,0,0,1,0,0,2017


In [12]:
# Aggregate discrete variables
quote_disc_df,aggregated_dict = aggregate_discrete(encoded_df,keepkeylist,encoded_dict) #DISCRETE AGGREGATION
del encoded_df
display(quote_disc_df)

,,requote_n,requote_y,requote_nan,policy_term_12,policy_term_6,policy_term_nan,tenuretype_new,tenuretype_nan,tenuresub_new,tenuresub_nan,...,farm_use_nan,large_farm_truck_n,large_farm_truck_y,large_farm_truck_nan,semi_farm_truck_n,semi_farm_truck_y,semi_farm_truck_nan,mulq_n,mulq_y,mulq_nan
trans_tracking_nbr,Year,,,,,,,,,,,,,,,,,,,,,
100000538,2019,1,0,0,0,1,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0
100000594,2019,1,0,0,0,1,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0
100000598,2019,1,0,0,0,1,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0
100000639,2019,1,0,0,0,1,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0
100000656,2019,1,0,0,0,1,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99997912,2019,1,0,0,0,1,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0
99997981,2019,1,0,0,0,1,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0
99997993,2019,1,0,0,0,1,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0


In [13]:
# Join discrete and continuous columns together
Curated_Quote = Curated_Quote.merge(quote_disc_df, on = keepkeylist, how = 'inner')
del quote_disc_df

In [14]:
# Display new curated primary dataset
#display(Curated_Quote)
# Export dataset to csv
Curated_Quote.to_csv("Curated_quote.csv", index=False)
# Export data types to json
with open("QuoteDict.json", "w") as outfile: 
    json.dump(Curated_Quote.dtypes.apply(lambda x: x.name).to_dict(), outfile)

In [15]:
# Extract mapping between trans_tracking_nbr and year (for appending year to secondary variables)
Trans_tracking_nbr_to_year_mapping = Curated_Quote[['trans_tracking_nbr', 'Year']].drop_duplicates()

## Fully Pre-Process Vehicle Dataset (Secondary File)

In [16]:
# Load json associated with primary file (ie. primarydict = load_json_dict('primary_dict.json'))
vehicle_dict = load_json_dict('col_types/vehicle_level_dict.json')
display(vehicle_dict)

{'annualmileage': 'continuous',
 'bipdsymbol': 'continuous',
 'collisionsymbol': 'continuous',
 'compsymbol': 'continuous',
 'medpipsymbol': 'continuous',
 'mileagetowork': 'continuous',
 'weeklytowork': 'continuous',
 'underinsuredmotoristspropertydamagelimit': 'continuous',
 'antilockbrakedevicecd': 'other',
 'bodystyle': 'other',
 'makecode': 'other',
 'modelliteral': 'other',
 'make': 'other',
 'model': 'other',
 'modelcode': 'other',
 'modelyear': 'other',
 'assignablevehind': 'discrete',
 'collisiondeductible': 'discrete',
 'communicationdeviceind': 'discrete',
 'comprehensivedeductible': 'discrete',
 'emergencyroadservice': 'discrete',
 'excessvehind': 'discrete',
 'factoryantitheftdeviceind': 'discrete',
 'highperformanceind': 'discrete',
 'safetyglass': 'discrete',
 'schoolchildrentransportation': 'discrete',
 'seasonalvehind': 'discrete',
 'snowplowliability': 'discrete',
 'usecd': 'discrete',
 'vehiclecategory': 'discrete',
 'vehicleownership': 'discrete',
 'vehicletype': 'd

In [17]:
# Import vehicle data from sample_quote_level.csv
vehicle_data = pd.read_csv("vehicle_level.csv")
#display(vehicle_data.columns)
set(vehicle_data.columns) - set(vehicle_dict)
set(vehicle_dict) - set(vehicle_data.columns)

set()

In [18]:
# Apply ad-hoc adjustments (ie. reduce number of discrete levels)
#quote_data["Year"] = quote_data["effdate"].apply(lambda X: str(X)[0:4])

In [19]:
# Cast columns to the correct type
for CurColumn in vehicle_dict.keys():
    if vehicle_dict[CurColumn] == 'discrete':
        vehicle_data[CurColumn] = vehicle_data[CurColumn].apply(str)
    elif vehicle_dict[CurColumn] == 'continuous':
        #vehicle_data[CurColumn] = vehicle_data[CurColumn].replace("None", np.nan)
        vehicle_data[CurColumn] = vehicle_data[CurColumn].apply(float)
    elif vehicle_dict[CurColumn] == 'key':
        vehicle_data[CurColumn] = vehicle_data[CurColumn].apply(str)
    else:
        pass
display(vehicle_data)

,trans_tracking_nbr,annualmileage,antilockbrakedevicecd,assignablevehind,bipdsymbol,bodystyle,collisiondeductible,collisionsymbol,communicationdeviceind,comprehensivedeductible,...,seasonalvehind,snowplowliability,usecd,vehiclecategory,vehicleownership,vehicletype,weeklytowork,underinsuredmotoristspropertydamagelimit,uninsuredmotoristspropertydamagedeductible,uninsuredmotoristspropertydamageind
0,65281610,NaN,NaN,Y,300.0,NaN,500,15.0,N,250,...,N,N,Pleasure,StandVeh,Owned,CarSuv,NaN,NaN,nan,N
1,65281610,NaN,NaN,Y,270.0,NaN,None,18.0,N,None,...,N,N,Pleasure,StandVeh,Owned,CarSuv,NaN,NaN,nan,N
2,65281610,NaN,NaN,Y,300.0,NaN,None,2.0,N,None,...,N,N,ToWork,StandVeh,Owned,Truck,200.0,NaN,nan,N
3,65384471,NaN,NaN,Y,295.0,NaN,500,32.0,N,500,...,N,N,Pleasure,StandVeh,Owned,Truck,NaN,NaN,nan,N
4,65283564,NaN,NaN,Y,295.0,NaN,500,24.0,N,500,...,N,N,ToWork,StandVeh,Owned,CarSuv,50.0,NaN,nan,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256463,174949272,NaN,NaN,Y,300.0,NaN,None,5.0,N,None,...,N,N,Pleasure,StandVeh,Owned,Truck,NaN,NaN,nan,nan
1256464,174949272,NaN,NaN,Y,310.0,NaN,None,11.0,N,None,...,N,N,ToWork,StandVeh,Owned,CarSuv,35.0,NaN,nan,nan
1256465,174949272,NaN,NaN,Y,300.0,NaN,500,20.0,N,250,...,N,N,Pleasure,StandVeh,Financed,Van,NaN,NaN,nan,nan
1256466,174949272,NaN,NaN,Y,295.0,NaN,500,11.0,N,250,...,N,N,Pleasure,StandVeh,Financed,Truck,NaN,NaN,nan,nan


In [20]:
# Append year to secondary dataset and its associated dictionary of data types
vehicle_data = vehicle_data.merge(Trans_tracking_nbr_to_year_mapping, on='trans_tracking_nbr', how='inner')
vehicle_dict["Year"] = "key"

In [21]:
# Ad Hoc -- Construct Vehicle Age from Vehicle Year and Year
#vehicle_data["modelyear"].isna().unique()
vehicle_data["vehicleage"] = vehicle_data["Year"].apply(float) - vehicle_data["modelyear"].apply(float)
vehicle_dict["vehicleage"] = 'continuous'
# Ad Hoc -- Remove model year (already used to construct vehicle age)
del vehicle_data["modelyear"]

In [22]:
# read vehiclecompany names from text document
makes_list = []
with open('output_make_txt.txt', 'r') as f:
    for line in f.readlines()[0:]:
        rows = line.rstrip('\n').split()
        makes_list.append(rows)
display(makes_list)

[['cheverolet',
  'cherrolet',
  'chevrolete',
  'chgevrolet',
  'chevrolet',
  'chevr',
  'chev',
  'chevy'],
 ['tissin', 'nissan'],
 ['jeepcj', 'cjjeep', 'jeep'],
 ['tyoga', 'taota', 'toyaota', 'toyata', 'toyota'],
 ['tcchrysler', 'chrysler'],
 ['polar',
  'poiaris',
  'poloris',
  'polairs',
  'polris',
  'poaris',
  'polaries',
  'polaris'],
 ['texas',
  'nexus',
  'exxs',
  'exis',
  'exciss',
  'exissii',
  'exxis',
  'exxiss',
  'exixx',
  'exiss',
  'exis3',
  'exifs'],
 ['harleydavison',
  'harlydavidson',
  'haeleydavidson',
  'harleydavidison',
  '79harleydavidson',
  'harleydavinson',
  'harleydavidscon',
  'harleydavidson',
  'harley',
  'harleydav',
  'harleydavis'],
 ['pontiacg6', 'ponti', 'pontiac'],
 ['volkswagon', 'volkswagen', 'volks'],
 ['mitzubishi', 'mitsibishi', 'mitsubishi', 'mitsc', 'mitsu'],
 ['merkur', 'merury', 'mercury'],
 ['plymoth', 'plymouth'],
 ['damon', 'diamondc', 'diamo', 'diamondd', 'diamondt', 'diamond', 'daymon'],
 ['dadillac', 'cadillac'],
 ['clu

In [23]:
# Make the dictionary to easily change values inside the makeliteral column
master_dict = {}
for lists in makes_list:
    for value in lists:
        master_dict[value] = lists[0]
display(master_dict)

{'cheverolet': 'cheverolet',
 'cherrolet': 'cheverolet',
 'chevrolete': 'cheverolet',
 'chgevrolet': 'cheverolet',
 'chevrolet': 'cheverolet',
 'chevr': 'cheverolet',
 'chev': 'cheverolet',
 'chevy': 'cheverolet',
 'tissin': 'tissin',
 'nissan': 'tissin',
 'jeepcj': 'jeepcj',
 'cjjeep': 'jeepcj',
 'jeep': 'jeepcj',
 'tyoga': 'tyoga',
 'taota': 'tyoga',
 'toyaota': 'tyoga',
 'toyata': 'tyoga',
 'toyota': 'tyoga',
 'tcchrysler': 'tcchrysler',
 'chrysler': 'tcchrysler',
 'polar': 'polar',
 'poiaris': 'polar',
 'poloris': 'polar',
 'polairs': 'polar',
 'polris': 'polar',
 'poaris': 'polar',
 'polaries': 'polar',
 'polaris': 'polar',
 'texas': 'texas',
 'nexus': 'texas',
 'exxs': 'texas',
 'exis': 'texas',
 'exciss': 'texas',
 'exissii': 'texas',
 'exxis': 'texas',
 'exxiss': 'texas',
 'exixx': 'texas',
 'exiss': 'texas',
 'exis3': 'texas',
 'exifs': 'texas',
 'harleydavison': 'harleydavison',
 'harlydavidson': 'harleydavison',
 'haeleydavidson': 'harleydavison',
 'harleydavidison': 'harley

In [24]:
#remove special characters from the makeliteral column
vehicle_data["makeliteral"] = vehicle_data["makeliteral"].apply(remove_special)
vehicle_data["makeliteral"].unique()

array(['chevrolet', 'ram', 'nissan', ..., 'hawkmaster', 'valor ez-go',
       'aplinelite'], dtype=object)

In [25]:
#replaces vehicle names with those on the makeliteral txt

# Better implementation
vehicle_data["makeliteral"] = vehicle_data["makeliteral"].apply(lambda X: master_dict[X] if X in master_dict else X)
display(vehicle_data["makeliteral"])


#for elem in makes_list:
#    for i in range(1, len(elem)):
#        vehicle_data['makeliteral'] = np.where(vehicle_data['makeliteral'] == elem[i], elem[0], vehicle_data['makeliteral'])
#display(vehicle_data['makeliteral'])
#
#Takes everything belo top 100 make literals and changes them to OTHER


0          cheverolet
1          cheverolet
2          cheverolet
3                 ram
4              tissin
              ...    
1256463          ford
1256464        merkur
1256465         dodge
1256466         dodge
1256467    cheverolet
Name: makeliteral, Length: 1256468, dtype: object

In [26]:
top100 = vehicle_data['makeliteral'].value_counts()[:100].index
vehicle_data.loc[~vehicle_data['makeliteral'].isin(top100), 'makeliteral'] = 'OTHER'

In [27]:
# Continuous variables -- impute missing values
# Extract list of continuous columns from dictionary
continuouscols = [col for col in vehicle_dict.keys() if vehicle_dict[col] == 'continuous']
# Construct dataframe containing means by year (for imputation)
#display(continuouscols)
########################
# Impute means by year #
########################
means_by_year = get_means_by_year(vehicle_data, continuouscols, "Year") 
# Impute missing continuous variables
vehicle_data = impute_missing_by_year(vehicle_data, continuouscols, "Year", means_by_year)
del means_by_year
##########################################
# Impute means across all years          #
# ... for when column not used in a year #
##########################################
# Construct dataframe containing overall means (for imputation)
means_overall = get_means_overall(vehicle_data, continuouscols)
# Impute missing continuous variables
vehicle_data = impute_missing_overall(vehicle_data, continuouscols, means_overall)
del means_overall

display(vehicle_data)

,trans_tracking_nbr,annualmileage,antilockbrakedevicecd,assignablevehind,bipdsymbol,bodystyle,collisiondeductible,collisionsymbol,communicationdeviceind,comprehensivedeductible,...,vehicleage,annualmileage_missing_ind,bipdsymbol_missing_ind,collisionsymbol_missing_ind,compsymbol_missing_ind,medpipsymbol_missing_ind,mileagetowork_missing_ind,weeklytowork_missing_ind,underinsuredmotoristspropertydamagelimit_missing_ind,vehicleage_missing_ind
0,65281610,2954.491724,NaN,Y,300.0,NaN,500,15.0,N,250,...,8.0,1,0,0,0,0,1,1,1,0
1,65281610,2954.491724,NaN,Y,270.0,NaN,None,18.0,N,None,...,18.0,1,0,0,0,0,1,1,1,0
2,65281610,2954.491724,NaN,Y,300.0,NaN,None,2.0,N,None,...,28.0,1,0,0,0,0,1,0,1,0
3,65384471,2954.491724,NaN,Y,295.0,NaN,500,32.0,N,500,...,3.0,1,0,0,0,0,1,1,1,0
4,65283564,2954.491724,NaN,Y,295.0,NaN,500,24.0,N,500,...,3.0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256463,3915185,1400.000000,NaN,Y,315.0,NaN,500,45.0,N,500,...,3.0,1,0,0,0,0,1,0,1,0
1256464,3915187,1400.000000,NaN,Y,285.0,NaN,1000,23.0,N,1000,...,2.0,1,0,0,0,0,1,0,1,0
1256465,3918187,1400.000000,NaN,N,300.0,NaN,None,16.0,N,100,...,14.0,1,0,0,0,0,1,1,1,0
1256466,3920186,1400.000000,NaN,Y,310.0,NaN,500,15.0,N,100,...,13.0,1,0,0,0,0,1,1,1,0


In [28]:
# Construct curated dataset
keepkeylist = list(set([col for col in vehicle_dict.keys() if vehicle_dict[col] == 'key']) -set(["primaryoperatorlink", "ratedoperatorlink"]))
# If modelyear converted to a key, don't forget to get rid of it
#keepkeylist = [col for col in vehicle_dict.keys() if vehicle_dict[col] == 'key']
display(keepkeylist)
Curated_Vehicle = vehicle_data[keepkeylist].drop_duplicates()
Curated_Vehicle

['trans_tracking_nbr', 'Year']

,trans_tracking_nbr,Year
0,65281610,2018
3,65384471,2018
4,65283564,2018
6,65383455,2018
8,65283567,2018
...,...,...
1256462,3914185,2014
1256463,3915185,2014
1256464,3915187,2014
1256465,3918187,2014


In [29]:
# Process continuous variables
cont_df = cont_agg(vehicle_data, continuouscols, keepkeylist)
Curated_Vehicle = Curated_Vehicle.merge(cont_df, on=keepkeylist, how='inner')
del cont_df
display(Curated_Vehicle)

,trans_tracking_nbr,Year,min_annualmileage,min_bipdsymbol,min_collisionsymbol,min_compsymbol,min_medpipsymbol,min_mileagetowork,min_underinsuredmotoristspropertydamagelimit,min_vehicleage,...,mean_weeklytowork,mean_annualmileage_missing_ind,mean_bipdsymbol_missing_ind,mean_collisionsymbol_missing_ind,mean_compsymbol_missing_ind,mean_medpipsymbol_missing_ind,mean_mileagetowork_missing_ind,mean_underinsuredmotoristspropertydamagelimit_missing_ind,mean_vehicleage_missing_ind,mean_weeklytowork_missing_ind
0,65281610,2018,2954.491724,270.0,2.0,2.0,475.0,NaN,NaN,8.0,...,111.416655,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.666667
1,65384471,2018,2954.491724,295.0,32.0,38.0,465.0,NaN,NaN,3.0,...,67.124982,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.000000
2,65283564,2018,2954.491724,295.0,22.0,21.0,465.0,NaN,NaN,2.0,...,95.000000,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.000000
3,65383455,2018,2954.491724,290.0,17.0,21.0,475.0,NaN,NaN,2.0,...,67.124982,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.000000
4,65283567,2018,2954.491724,290.0,3.0,3.0,485.0,NaN,NaN,6.0,...,56.593737,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717455,3914185,2014,1400.000000,310.0,11.0,11.0,480.0,NaN,NaN,6.0,...,100.000000,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.000000
717456,3915185,2014,1400.000000,315.0,45.0,22.0,530.0,NaN,NaN,3.0,...,400.000000,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.000000
717457,3915187,2014,1400.000000,285.0,23.0,19.0,490.0,NaN,NaN,2.0,...,200.000000,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.000000
717458,3918187,2014,1400.000000,300.0,16.0,16.0,500.0,NaN,NaN,14.0,...,46.651264,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.000000


In [30]:
# Process discrete variables
discretecols = [col for col in vehicle_dict.keys() if vehicle_dict[col] == 'discrete']
encoded_df, encoded_dict = One_Hot_Encode(vehicle_data, discretecols, keepkeylist)
encoded_df

,trans_tracking_nbr,assignablevehind_n,assignablevehind_y,assignablevehind_nan,collisiondeductible_100,collisiondeductible_1000,collisiondeductible_200,collisiondeductible_2000,collisiondeductible_250,collisiondeductible_50,...,vehicletype_nan,uninsuredmotoristspropertydamagedeductible_100,uninsuredmotoristspropertydamagedeductible_200,uninsuredmotoristspropertydamagedeductible_250,uninsuredmotoristspropertydamagedeductible_nodeductible,uninsuredmotoristspropertydamagedeductible_nan,uninsuredmotoristspropertydamageind_n,uninsuredmotoristspropertydamageind_y,uninsuredmotoristspropertydamageind_nan,Year
0,65281610,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,2018
1,65281610,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,2018
2,65281610,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,2018
3,65384471,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,2018
4,65283564,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256463,3915185,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,2014
1256464,3915187,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,0,2014
1256465,3918187,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,2014
1256466,3920186,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,2014


In [31]:
# Aggregate discrete variables
vehicle_disc_df,aggregated_dict = aggregate_discrete(encoded_df,keepkeylist,encoded_dict) #DISCRETE AGGREGATION
del encoded_df
display(vehicle_disc_df)

,,assignablevehind_n,assignablevehind_y,assignablevehind_nan,collisiondeductible_100,collisiondeductible_1000,collisiondeductible_200,collisiondeductible_2000,collisiondeductible_250,collisiondeductible_50,collisiondeductible_500,...,vehicletype_vanbus,vehicletype_nan,uninsuredmotoristspropertydamagedeductible_100,uninsuredmotoristspropertydamagedeductible_200,uninsuredmotoristspropertydamagedeductible_250,uninsuredmotoristspropertydamagedeductible_nodeductible,uninsuredmotoristspropertydamagedeductible_nan,uninsuredmotoristspropertydamageind_n,uninsuredmotoristspropertydamageind_y,uninsuredmotoristspropertydamageind_nan
trans_tracking_nbr,Year,,,,,,,,,,,,,,,,,,,,,
100000538,2019,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,...,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
100000594,2019,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
100000598,2019,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,...,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
100000639,2019,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,...,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
100000656,2019,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,...,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99997912,2019,0.0,1.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
99997981,2019,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,...,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
99997993,2019,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [32]:
# Join discrete and continuous columns together
Curated_Vehicle = Curated_Vehicle.merge(vehicle_disc_df, on = keepkeylist, how = 'inner')
del vehicle_disc_df
display(Curated_Vehicle)

,trans_tracking_nbr,Year,min_annualmileage,min_bipdsymbol,min_collisionsymbol,min_compsymbol,min_medpipsymbol,min_mileagetowork,min_underinsuredmotoristspropertydamagelimit,min_vehicleage,...,vehicletype_vanbus,vehicletype_nan,uninsuredmotoristspropertydamagedeductible_100,uninsuredmotoristspropertydamagedeductible_200,uninsuredmotoristspropertydamagedeductible_250,uninsuredmotoristspropertydamagedeductible_nodeductible,uninsuredmotoristspropertydamagedeductible_nan,uninsuredmotoristspropertydamageind_n,uninsuredmotoristspropertydamageind_y,uninsuredmotoristspropertydamageind_nan
0,65281610,2018,2954.491724,270.0,2.0,2.0,475.0,NaN,NaN,8.0,...,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,65384471,2018,2954.491724,295.0,32.0,38.0,465.0,NaN,NaN,3.0,...,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,65283564,2018,2954.491724,295.0,22.0,21.0,465.0,NaN,NaN,2.0,...,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,65383455,2018,2954.491724,290.0,17.0,21.0,475.0,NaN,NaN,2.0,...,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,65283567,2018,2954.491724,290.0,3.0,3.0,485.0,NaN,NaN,6.0,...,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717455,3914185,2014,1400.000000,310.0,11.0,11.0,480.0,NaN,NaN,6.0,...,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
717456,3915185,2014,1400.000000,315.0,45.0,22.0,530.0,NaN,NaN,3.0,...,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
717457,3915187,2014,1400.000000,285.0,23.0,19.0,490.0,NaN,NaN,2.0,...,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
717458,3918187,2014,1400.000000,300.0,16.0,16.0,500.0,NaN,NaN,14.0,...,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [33]:
# Add count of secondary key
secondary_count_df = count_distinct(vehicle_data, keepkeylist, 'vehiclecount')
Curated_Vehicle = Curated_Vehicle.merge(secondary_count_df, on=keepkeylist, how='inner')
del secondary_count_df
Curated_Vehicle

,trans_tracking_nbr,Year,min_annualmileage,min_bipdsymbol,min_collisionsymbol,min_compsymbol,min_medpipsymbol,min_mileagetowork,min_underinsuredmotoristspropertydamagelimit,min_vehicleage,...,vehicletype_nan,uninsuredmotoristspropertydamagedeductible_100,uninsuredmotoristspropertydamagedeductible_200,uninsuredmotoristspropertydamagedeductible_250,uninsuredmotoristspropertydamagedeductible_nodeductible,uninsuredmotoristspropertydamagedeductible_nan,uninsuredmotoristspropertydamageind_n,uninsuredmotoristspropertydamageind_y,uninsuredmotoristspropertydamageind_nan,vehiclecount
0,65281610,2018,2954.491724,270.0,2.0,2.0,475.0,NaN,NaN,8.0,...,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,3
1,65384471,2018,2954.491724,295.0,32.0,38.0,465.0,NaN,NaN,3.0,...,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1
2,65283564,2018,2954.491724,295.0,22.0,21.0,465.0,NaN,NaN,2.0,...,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
3,65383455,2018,2954.491724,290.0,17.0,21.0,475.0,NaN,NaN,2.0,...,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2
4,65283567,2018,2954.491724,290.0,3.0,3.0,485.0,NaN,NaN,6.0,...,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717455,3914185,2014,1400.000000,310.0,11.0,11.0,480.0,NaN,NaN,6.0,...,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1
717456,3915185,2014,1400.000000,315.0,45.0,22.0,530.0,NaN,NaN,3.0,...,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1
717457,3915187,2014,1400.000000,285.0,23.0,19.0,490.0,NaN,NaN,2.0,...,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1
717458,3918187,2014,1400.000000,300.0,16.0,16.0,500.0,NaN,NaN,14.0,...,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1


In [34]:
Curated_Vehicle.to_csv("Curated_Vehicle.csv")

## Fully Pre-Process Operator Dataset (Secondary File)

In [35]:
operator_dict = load_json_dict('col_types/operator_level_dict.json')
display(operator_dict)

{'personsage': 'continuous',
 'yearslicensed': 'continuous',
 'dateofbirth': 'other',
 'ratinganalysisoperatorclasscode': 'other',
 'licensestate': 'other',
 'distancetoschool': 'discrete',
 'accesstovehicleind': 'discrete',
 'activedutyalertind': 'discrete',
 'arrestedorconvicted': 'discrete',
 'awayatschoolind': 'discrete',
 'defensivedrivcourseind': 'discrete',
 'educationstatus': 'discrete',
 'engagedind': 'discrete',
 'fulltimestudent': 'discrete',
 'gender': 'discrete',
 'goodstudentcert': 'discrete',
 'gradepointavg': 'discrete',
 'identitytheftvictimalertind': 'discrete',
 'occupation': 'discrete',
 'operatorstatus': 'discrete',
 'relationship': 'discrete',
 'retiredind': 'discrete',
 'trans_tracking_nbr': 'key',
 'clientid': 'key',
 'residentoperatorlink': 'key'}

In [36]:
# Import vehicle data from sample_quote_level.csv
operator_data = pd.read_csv("operator_level.csv")
#display(vehicle_data.columns)
#print("1")
set(operator_data.columns) - set(operator_dict)
#print("2")
set(operator_dict) - set(operator_data.columns)

set()

In [37]:
# Apply ad-hoc adjustments (ie. reduce number of discrete levels)
#quote_data["Year"] = quote_data["effdate"].apply(lambda X: str(X)[0:4])

In [38]:
# Cast columns to the correct type
for CurColumn in operator_dict.keys():
    if operator_dict[CurColumn] == 'discrete':
        operator_data[CurColumn] = operator_data[CurColumn].apply(str)
    elif operator_dict[CurColumn] == 'continuous':
        operator_data[CurColumn] = operator_data[CurColumn].apply(float)
    elif operator_dict[CurColumn] == 'key':
        operator_data[CurColumn] = operator_data[CurColumn].apply(str)
    else:
        pass
display(operator_data)

,trans_tracking_nbr,accesstovehicleind,activedutyalertind,arrestedorconvicted,awayatschoolind,clientid,dateofbirth,defensivedrivcourseind,distancetoschool,educationstatus,...,licensestate,occupation,operatorstatus,personsage,ratinganalysisoperatorclasscode,relationship,retiredind,residentoperatorlink,yearslicensed,duplicate_clientid?
0,106057351,nan,N,nan,nan,RK7Y2bU8mJz2skx,5/8/1982,nan,nan,nan,...,IL,Other,Active,36.0,A7,SINGLE,nan,16182101.0,20.0,False
1,106057355,nan,nan,nan,nan,RP7XWLJ8lQzA0u4,10/13/1964,None,nan,nan,...,OR,Other,Active,54.0,A3,MARRIED,nan,322773101.0,38.0,False
2,106057355,nan,nan,nan,nan,RP8aCmk8mbz230i,5/18/1963,None,nan,nan,...,OR,Other,Active,56.0,A7,MARRIED,nan,222773101.0,40.0,False
3,106057309,nan,nan,nan,nan,RP7sr728kjz1IYm,3/20/1995,None,nan,None,...,OR,Other,Active,24.0,C4,SINGLE,nan,891773101.0,8.0,False
4,106002666,nan,nan,nan,nan,RP7sr728mpz1IYm,6/10/1970,N,nan,nan,...,GA,Other,Active,48.0,A7,SINGLE,nan,739573101.0,32.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,158383723,nan,nan,nan,nan,S05Vf5l8lQz1UML,6/19/1952,N,nan,nan,...,NV,Other,Active,67.0,A4,SINGLE,nan,465089101.0,51.0,False
1048571,158389831,nan,nan,nan,nan,qaR18d36f3Y7801,4/12/1952,N,nan,nan,...,GA,Other,Active,68.0,A8,MARRIED,nan,356089101.0,52.0,False
1048572,158389831,nan,nan,nan,nan,S05KQKx8lwz1Si6,5/19/1942,N,nan,nan,...,GA,Other,Active,78.0,A4,MARRIED,nan,256089101.0,62.0,False
1048573,158389842,nan,nan,nan,nan,Tna817z3Hnxk645,1/1/1987,nan,nan,nan,...,MN,Other,Active,33.0,A3,SINGLE,nan,166089101.0,17.0,False


In [39]:
# Append year to secondary dataset and its associated dictionary of data types
operator_data = operator_data.merge(Trans_tracking_nbr_to_year_mapping, on='trans_tracking_nbr', how='inner')
operator_dict["Year"] = "key"

In [40]:
 # Continuous variables -- impute missing values
# Extract list of continuous columns from dictionary
continuouscols = [col for col in operator_dict.keys() if operator_dict[col] == 'continuous']
# Construct dataframe containing means by year (for imputation)
#display(continuouscols)
########################
# Impute means by year #
########################
means_by_year = get_means_by_year(operator_data, continuouscols, "Year") 
# Impute missing continuous variables
operator_data = impute_missing_by_year(operator_data, continuouscols, "Year", means_by_year)
del means_by_year
##########################################
# Impute means across all years          #
# ... for when column not used in a year #
##########################################
# Construct dataframe containing overall means (for imputation)
means_overall = get_means_overall(operator_data, continuouscols)
# Impute missing continuous variables
operator_data = impute_missing_overall(operator_data, continuouscols, means_overall)
del means_overall
display(operator_data)

,trans_tracking_nbr,accesstovehicleind,activedutyalertind,arrestedorconvicted,awayatschoolind,clientid,dateofbirth,defensivedrivcourseind,distancetoschool,educationstatus,...,personsage,ratinganalysisoperatorclasscode,relationship,retiredind,residentoperatorlink,yearslicensed,duplicate_clientid?,Year,personsage_missing_ind,yearslicensed_missing_ind
0,106057351,nan,N,nan,nan,RK7Y2bU8mJz2skx,5/8/1982,nan,nan,nan,...,36.0,A7,SINGLE,nan,16182101.0,20.0,False,2019,0,0
1,106057355,nan,nan,nan,nan,RP7XWLJ8lQzA0u4,10/13/1964,None,nan,nan,...,54.0,A3,MARRIED,nan,322773101.0,38.0,False,2019,0,0
2,106057355,nan,nan,nan,nan,RP8aCmk8mbz230i,5/18/1963,None,nan,nan,...,56.0,A7,MARRIED,nan,222773101.0,40.0,False,2019,0,0
3,106057309,nan,nan,nan,nan,RP7sr728kjz1IYm,3/20/1995,None,nan,None,...,24.0,C4,SINGLE,nan,891773101.0,8.0,False,2019,0,0
4,106002666,nan,nan,nan,nan,RP7sr728mpz1IYm,6/10/1970,N,nan,nan,...,48.0,A7,SINGLE,nan,739573101.0,32.0,False,2019,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,187824281,nan,nan,nan,nan,SJQ8V5q8lez1LmK,7/17/1974,nan,nan,nan,...,46.0,A3,MARRIED,nan,295882201.0,30.0,False,2021,0,0
1048571,187820351,nan,nan,nan,nan,qdk1B94Fqwx5808,6/7/1986,nan,nan,nan,...,34.0,A7,MARRIED,nan,960782201.0,18.0,False,2021,0,0
1048572,187820351,nan,nan,nan,nan,SHG24fy8klzaPOK,9/27/1983,nan,nan,nan,...,37.0,A3,MARRIED,nan,860782201.0,21.0,False,2021,0,0
1048573,187821346,nan,nan,nan,nan,LPIxol3DIs31816,9/23/1979,nan,nan,nan,...,41.0,A7,MARRIED,nan,975882201.0,25.0,False,2021,0,0


In [41]:
# Construct curated dataset
keepkeylist = list(set([col for col in operator_dict.keys() if operator_dict[col] == 'key']) - set(["residentoperatorlink","clientid"]))
# If modelyear converted to a key, don't forget to get rid of it
#keepkeylist = [col for col in vehicle_dict.keys() if vehicle_dict[col] == 'key']
display(keepkeylist)
Curated_Operator = operator_data[keepkeylist].drop_duplicates()
Curated_Operator

['trans_tracking_nbr', 'Year']

,trans_tracking_nbr,Year
0,106057351,2019
1,106057355,2019
3,106057309,2019
4,106002666,2019
5,106055490,2019
...,...,...
1048567,187782644,2021
1048568,187785846,2021
1048569,187824281,2021
1048571,187820351,2021


In [42]:
# Process continuous variables
cont_df = cont_agg(operator_data, continuouscols, keepkeylist)
Curated_Operator = Curated_Operator.merge(cont_df, on=keepkeylist, how='inner')
del cont_df
display(Curated_Operator)

,trans_tracking_nbr,Year,min_personsage,min_yearslicensed,max_personsage,max_yearslicensed,mean_personsage,mean_yearslicensed,mean_personsage_missing_ind,mean_yearslicensed_missing_ind
0,106057351,2019,36.0,20.0,36.0,20.0,36.0,20.0,0.0,0.0
1,106057355,2019,54.0,38.0,56.0,40.0,55.0,39.0,0.0,0.0
2,106057309,2019,24.0,8.0,24.0,8.0,24.0,8.0,0.0,0.0
3,106002666,2019,48.0,32.0,48.0,32.0,48.0,32.0,0.0,0.0
4,106055490,2019,18.0,2.0,35.0,19.0,26.5,10.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
656317,187782644,2021,37.0,21.0,37.0,21.0,37.0,21.0,0.0,0.0
656318,187785846,2021,76.0,60.0,76.0,60.0,76.0,60.0,0.0,0.0
656319,187824281,2021,46.0,30.0,50.0,34.0,48.0,32.0,0.0,0.0
656320,187820351,2021,34.0,18.0,37.0,21.0,35.5,19.5,0.0,0.0


In [43]:
 # Process discrete variables
discretecols = [col for col in operator_dict.keys() if operator_dict[col] == 'discrete']
encoded_df, encoded_dict = One_Hot_Encode(operator_data, discretecols, keepkeylist)
encoded_df

,trans_tracking_nbr,accesstovehicleind_n,accesstovehicleind_y,accesstovehicleind_nan,activedutyalertind_n,activedutyalertind_y,activedutyalertind_nan,arrestedorconvicted,awayatschoolind_n,awayatschoolind_y,...,operatorstatus_inactiveoperator,operatorstatus_nonoperator,operatorstatus_nan,relationship_civunion,relationship_domesticpartner,relationship_married,relationship_single,relationship_nan,retiredind,Year
0,106057351,0,0,1,1,0,0,NaN,0,0,...,0,0,0,0,0,0,1,0,NaN,2019
1,106057355,0,0,1,0,0,1,NaN,0,0,...,0,0,0,0,0,1,0,0,NaN,2019
2,106057355,0,0,1,0,0,1,NaN,0,0,...,0,0,0,0,0,1,0,0,NaN,2019
3,106057309,0,0,1,0,0,1,NaN,0,0,...,0,0,0,0,0,0,1,0,NaN,2019
4,106002666,0,0,1,0,0,1,NaN,0,0,...,0,0,0,0,0,0,1,0,NaN,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,187824281,0,0,1,0,0,1,NaN,0,0,...,0,0,0,0,0,1,0,0,NaN,2021
1048571,187820351,0,0,1,0,0,1,NaN,0,0,...,0,0,0,0,0,1,0,0,NaN,2021
1048572,187820351,0,0,1,0,0,1,NaN,0,0,...,0,0,0,0,0,1,0,0,NaN,2021
1048573,187821346,0,0,1,0,0,1,NaN,0,0,...,0,0,0,0,0,1,0,0,NaN,2021


In [44]:
# Aggregate discrete variables
operator_disc_df,aggregated_dict = aggregate_discrete(encoded_df,keepkeylist,encoded_dict) #DISCRETE AGGREGATION
del encoded_df
display(operator_disc_df)

,,accesstovehicleind_n,accesstovehicleind_y,accesstovehicleind_nan,activedutyalertind_n,activedutyalertind_y,activedutyalertind_nan,arrestedorconvicted,awayatschoolind_n,awayatschoolind_y,awayatschoolind_nan,...,operatorstatus_addedfromprop,operatorstatus_inactiveoperator,operatorstatus_nonoperator,operatorstatus_nan,relationship_civunion,relationship_domesticpartner,relationship_married,relationship_single,relationship_nan,retiredind
trans_tracking_nbr,Year,,,,,,,,,,,,,,,,,,,,,
100000538,2019,0.0,0.0,1.0,1.000000,0.0,0.000000,NaN,0.000000,0.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN
100000594,2019,0.0,0.0,1.0,1.000000,0.0,0.000000,NaN,0.000000,0.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
100000598,2019,0.0,0.0,1.0,1.000000,0.0,0.000000,NaN,0.000000,0.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
100000639,2019,0.0,0.0,1.0,1.000000,0.0,0.000000,NaN,0.000000,0.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
100000656,2019,0.0,0.0,1.0,1.000000,0.0,0.000000,NaN,0.000000,0.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99997912,2019,0.0,0.0,1.0,1.000000,0.0,0.000000,NaN,0.000000,0.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN
99997981,2019,0.0,0.0,1.0,1.000000,0.0,0.000000,NaN,0.000000,0.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN
99997993,2019,0.0,0.0,1.0,1.000000,0.0,0.000000,NaN,0.000000,0.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN


In [45]:
# Join discrete and continuous columns together
Curated_Operator = Curated_Operator.merge(operator_disc_df, on = keepkeylist, how = 'inner')
del operator_disc_df
display(Curated_Operator)

,trans_tracking_nbr,Year,min_personsage,min_yearslicensed,max_personsage,max_yearslicensed,mean_personsage,mean_yearslicensed,mean_personsage_missing_ind,mean_yearslicensed_missing_ind,...,operatorstatus_addedfromprop,operatorstatus_inactiveoperator,operatorstatus_nonoperator,operatorstatus_nan,relationship_civunion,relationship_domesticpartner,relationship_married,relationship_single,relationship_nan,retiredind
0,106057351,2019,36.0,20.0,36.0,20.0,36.0,20.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
1,106057355,2019,54.0,38.0,56.0,40.0,55.0,39.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN
2,106057309,2019,24.0,8.0,24.0,8.0,24.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
3,106002666,2019,48.0,32.0,48.0,32.0,48.0,32.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
4,106055490,2019,18.0,2.0,35.0,19.0,26.5,10.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656317,187782644,2021,37.0,21.0,37.0,21.0,37.0,21.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
656318,187785846,2021,76.0,60.0,76.0,60.0,76.0,60.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
656319,187824281,2021,46.0,30.0,50.0,34.0,48.0,32.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN
656320,187820351,2021,34.0,18.0,37.0,21.0,35.5,19.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN


In [46]:
# Add count of secondary key
secondary_count_df = count_distinct(operator_data, keepkeylist, 'operatorcount')
Curated_Operator = Curated_Operator.merge(secondary_count_df, on=keepkeylist, how='inner')
del secondary_count_df
Curated_Operator

,trans_tracking_nbr,Year,min_personsage,min_yearslicensed,max_personsage,max_yearslicensed,mean_personsage,mean_yearslicensed,mean_personsage_missing_ind,mean_yearslicensed_missing_ind,...,operatorstatus_inactiveoperator,operatorstatus_nonoperator,operatorstatus_nan,relationship_civunion,relationship_domesticpartner,relationship_married,relationship_single,relationship_nan,retiredind,operatorcount
0,106057351,2019,36.0,20.0,36.0,20.0,36.0,20.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,1
1,106057355,2019,54.0,38.0,56.0,40.0,55.0,39.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,2
2,106057309,2019,24.0,8.0,24.0,8.0,24.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,1
3,106002666,2019,48.0,32.0,48.0,32.0,48.0,32.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,1
4,106055490,2019,18.0,2.0,35.0,19.0,26.5,10.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656317,187782644,2021,37.0,21.0,37.0,21.0,37.0,21.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,1
656318,187785846,2021,76.0,60.0,76.0,60.0,76.0,60.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,1
656319,187824281,2021,46.0,30.0,50.0,34.0,48.0,32.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,2
656320,187820351,2021,34.0,18.0,37.0,21.0,35.5,19.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,2


In [47]:
Curated_Operator.to_csv("Curated_Operator.csv")

### Merge Datasets Together

In [48]:
Merged = Curated_Quote.merge(Curated_Vehicle, on=keepkeylist, how ='inner').merge(Curated_Operator, on=keepkeylist, how ='inner')
del Curated_Quote, Curated_Operator, Curated_Vehicle
Merged

,trans_tracking_nbr,Year,min_age_oldest_driver,min_age_youngest_driver,min_cbr,min_cbr_insufficientcredit_ind,min_cbr_nohit_ind,min_cbrscore,min_chargeable13,min_chargeable45,...,operatorstatus_inactiveoperator,operatorstatus_nonoperator,operatorstatus_nan,relationship_civunion,relationship_domesticpartner,relationship_married,relationship_single,relationship_nan,retiredind,operatorcount
0,806186,2014,29.0,28.0,3.256082,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,NaN,2
1,3437185,2014,53.0,53.0,3.256082,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,NaN,2
2,3440190,2014,34.0,34.0,3.256082,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,NaN,1
3,3441186,2014,48.0,48.0,3.256082,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,NaN,2
4,3442186,2014,54.0,54.0,3.256082,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656317,47433200,2017,67.0,67.0,9.000000,0.0,0.0,522.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,NaN,1
656318,38302236,2017,40.0,33.0,3.501901,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,NaN,2
656319,38287210,2017,26.0,26.0,3.501901,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,NaN,1
656320,38302245,2017,22.0,22.0,3.501901,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,NaN,1


## Normalize columns

In [49]:
# Specify columns to normalize by year (all other columns are normalized overall / across all years)
NormalizeByYear = [col for col in Merged.columns if col not in keepkeylist and 'premium' in col]

In [50]:
# Normalizing columns overall
columns = [col for col in Merged.columns if col not in keepkeylist and col not in NormalizeByYear]
print("Normalizing columns {}".format(columns))
print("Merged results prior to normalization have dimensions {}".format(Merged.shape))
Merged = normalize_data(Merged, columns)
print("Merged results after normalization have dimensions {}".format(Merged.shape))
#display(time.time() - initial)
display(Merged)

Normalizing columns ['min_age_oldest_driver', 'min_age_youngest_driver', 'min_cbr', 'min_cbr_insufficientcredit_ind', 'min_cbr_nohit_ind', 'min_cbrscore', 'min_chargeable13', 'min_chargeable45', 'min_drivers', 'min_newest_vehicle', 'min_oldest_vehicle', 'min_pd_limit', 'min_qtever', 'min_vehicles', 'max_age_oldest_driver', 'max_age_youngest_driver', 'max_cbr', 'max_cbr_insufficientcredit_ind', 'max_cbr_nohit_ind', 'max_cbrscore', 'max_chargeable13', 'max_chargeable45', 'max_drivers', 'max_newest_vehicle', 'max_oldest_vehicle', 'max_pd_limit', 'max_qtever', 'max_vehicles', 'mean_age_oldest_driver', 'mean_age_youngest_driver', 'mean_cbr', 'mean_cbr_insufficientcredit_ind', 'mean_cbr_nohit_ind', 'mean_cbrscore', 'mean_chargeable13', 'mean_chargeable45', 'mean_drivers', 'mean_newest_vehicle', 'mean_oldest_vehicle', 'mean_pd_limit', 'mean_qtever', 'mean_vehicles', 'mean_age_oldest_driver_missing_ind', 'mean_age_youngest_driver_missing_ind', 'mean_cbr_insufficientcredit_ind_missing_ind', 'me

c:\users\id75147\appdata\local\programs\python\python39\lib\site-packages\sklearn\preprocessing\_data.py:179: RuntimeWarning: Mean of empty slice
  mean_ = np.nanmean(X, axis)
c:\users\id75147\appdata\local\programs\python\python39\lib\site-packages\numpy\lib\nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\users\id75147\appdata\local\programs\python\python39\lib\site-packages\sklearn\preprocessing\_data.py:187: RuntimeWarning: Mean of empty slice
  mean_1 = np.nanmean(Xr, axis=0)
c:\users\id75147\appdata\local\programs\python\python39\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
c:\users\id75147\appdata\local\programs\python\python39\lib\site-packages\sklearn\prepro

c:\users\id75147\appdata\local\programs\python\python39\lib\site-packages\sklearn\preprocessing\_data.py:179: RuntimeWarning: Mean of empty slice
  mean_ = np.nanmean(X, axis)
c:\users\id75147\appdata\local\programs\python\python39\lib\site-packages\numpy\lib\nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\users\id75147\appdata\local\programs\python\python39\lib\site-packages\sklearn\preprocessing\_data.py:187: RuntimeWarning: Mean of empty slice
  mean_1 = np.nanmean(Xr, axis=0)
c:\users\id75147\appdata\local\programs\python\python39\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
c:\users\id75147\appdata\local\programs\python\python39\lib\site-packages\sklearn\prepro

Merged results after normalization have dimensions (656322, 728)


,trans_tracking_nbr,Year,min_age_oldest_driver,min_age_youngest_driver,min_cbr,min_cbr_insufficientcredit_ind,min_cbr_nohit_ind,min_cbrscore,min_chargeable13,min_chargeable45,...,operatorstatus_inactiveoperator,operatorstatus_nonoperator,operatorstatus_nan,relationship_civunion,relationship_domesticpartner,relationship_married,relationship_single,relationship_nan,retiredind,operatorcount
0,806186,2014,-0.984062,-0.232180,-0.005260,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,1.372892,-1.361077,-0.014038,NaN,0.528600
1,3437185,2014,0.473097,0.061800,-0.005260,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,1.372892,-1.361077,-0.014038,NaN,0.528600
2,3440190,2014,-0.680488,-0.161625,-0.005260,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,-0.789905,0.796782,-0.014038,NaN,-0.785192
3,3441186,2014,0.169522,0.003004,-0.005260,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,1.372892,-1.361077,-0.014038,NaN,0.528600
4,3442186,2014,0.533812,0.073559,-0.005260,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,-0.789905,0.796782,-0.014038,NaN,-0.785192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656317,47433200,2017,1.323106,0.226429,2.570008,-0.176531,-0.113304,0.024326,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,-0.789905,0.796782,-0.014038,NaN,-0.785192
656318,38302236,2017,-0.316198,-0.173384,0.104952,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,1.372892,-1.361077,-0.014038,NaN,0.528600
656319,38287210,2017,-1.166207,-0.255699,0.104952,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,-0.789905,0.796782,-0.014038,NaN,-0.785192
656320,38302245,2017,-1.409067,-0.302735,0.104952,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,-0.789905,0.796782,-0.014038,NaN,-0.785192


In [51]:
# Normalize columns by year
columns = [col for col in Merged.columns if col not in keepkeylist and col in NormalizeByYear]
print("Normalizing columns {}".format(columns))
print("Merged results prior to normalization have dimensions {}".format(Merged.shape))
Merged = normalize_data(Merged, columns, year_col = 'Year')
print("Merged results after normalization have dimensions {}".format(Merged.shape))
display(Merged)

Normalizing columns ['min_premium', 'max_premium', 'mean_premium', 'mean_premium_missing_ind']
Merged results prior to normalization have dimensions (656322, 728)
Merged results after normalization have dimensions (656322, 728)


,trans_tracking_nbr,Year,min_age_oldest_driver,min_age_youngest_driver,min_cbr,min_cbr_insufficientcredit_ind,min_cbr_nohit_ind,min_cbrscore,min_chargeable13,min_chargeable45,...,operatorstatus_inactiveoperator,operatorstatus_nonoperator,operatorstatus_nan,relationship_civunion,relationship_domesticpartner,relationship_married,relationship_single,relationship_nan,retiredind,operatorcount
0,806186,2014,-0.984062,-0.232180,-0.005260,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,1.372892,-1.361077,-0.014038,NaN,0.528600
1,3437185,2014,0.473097,0.061800,-0.005260,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,1.372892,-1.361077,-0.014038,NaN,0.528600
2,3440190,2014,-0.680488,-0.161625,-0.005260,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,-0.789905,0.796782,-0.014038,NaN,-0.785192
3,3441186,2014,0.169522,0.003004,-0.005260,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,1.372892,-1.361077,-0.014038,NaN,0.528600
4,3442186,2014,0.533812,0.073559,-0.005260,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,-0.789905,0.796782,-0.014038,NaN,-0.785192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656317,47433200,2017,1.323106,0.226429,2.570008,-0.176531,-0.113304,0.024326,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,-0.789905,0.796782,-0.014038,NaN,-0.785192
656318,38302236,2017,-0.316198,-0.173384,0.104952,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,1.372892,-1.361077,-0.014038,NaN,0.528600
656319,38287210,2017,-1.166207,-0.255699,0.104952,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,-0.789905,0.796782,-0.014038,NaN,-0.785192
656320,38302245,2017,-1.409067,-0.302735,0.104952,-0.176531,-0.113304,-1.693483,-0.504462,-0.303277,...,-0.049345,-0.149307,-0.013952,-0.016481,-0.061487,-0.789905,0.796782,-0.014038,NaN,-0.785192


In [52]:
# Dropping columns that have 100% NaN values
for col in Merged.columns:
    if Merged[col].isnull().any() or len(Merged[col].unique())<=1:
        Merged.drop(col, inplace = True, axis = 1)

In [53]:
# Construct data dictionary (keys mapped as 'str', values to use in auto-encoder mapped as 'float')
DataDictionary = {X:'str' if X in keepkeylist else 'float' for X in Merged.columns}
DataDictionary

{'trans_tracking_nbr': 'str',
 'Year': 'str',
 'min_age_oldest_driver': 'float',
 'min_age_youngest_driver': 'float',
 'min_cbr': 'float',
 'min_cbr_insufficientcredit_ind': 'float',
 'min_cbr_nohit_ind': 'float',
 'min_cbrscore': 'float',
 'min_chargeable13': 'float',
 'min_chargeable45': 'float',
 'min_drivers': 'float',
 'min_newest_vehicle': 'float',
 'min_oldest_vehicle': 'float',
 'min_pd_limit': 'float',
 'min_premium': 'float',
 'min_qtever': 'float',
 'min_vehicles': 'float',
 'max_age_oldest_driver': 'float',
 'max_age_youngest_driver': 'float',
 'max_cbr': 'float',
 'max_cbr_insufficientcredit_ind': 'float',
 'max_cbr_nohit_ind': 'float',
 'max_cbrscore': 'float',
 'max_chargeable13': 'float',
 'max_chargeable45': 'float',
 'max_drivers': 'float',
 'max_newest_vehicle': 'float',
 'max_oldest_vehicle': 'float',
 'max_pd_limit': 'float',
 'max_premium': 'float',
 'max_qtever': 'float',
 'max_vehicles': 'float',
 'mean_age_oldest_driver': 'float',
 'mean_age_youngest_driver': '

## Export Results

In [54]:
# Export Merged Dataset
Merged.to_csv("Normalized_df.csv", index = False)

In [55]:
# Export data dictionary
with open("Normalized_datadictionary.json", "w") as outfile: 
    json.dump(DataDictionary, outfile)

In [56]:
display(time.time() - initial)

1108.98690533638